In [1]:
# gpu number setting
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '1' ## gpu 번호 셋팅 윤건 :0, 기준 : 1, 준형 :2,

# Gpu growth setting
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


# tensorflow & keras version check
print('tensorflow version : ' , tf.__version__)
print('keras version : ' , tf.keras.__version__)

# tensorflow gpu available check 
print('GPU available ? : ', tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tensorflow version :  2.3.1
keras version :  2.4.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU available ? :  True


In [2]:
import pandas as pd
import numpy as np
import pickle
import tqdm
import random
import time

In [3]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [4]:
with open('../../Data/movie_201118_table_5.pickle', 'rb') as f:
    raw = pickle.load(f)

* final drop

In [5]:
num_genre = 33

In [6]:
sparse_features = ['sa_id', 'album_id', 'buy_seg'] + [str(i) for i in range(num_genre)]
dense_features = ['release_date', 'run_time', 'movie_meta_price', 'i30_meta_price', 'buy_tot',
                  'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4']
unnecessary_features = ['index', 'meta_genre', 'genre_large', 'genre_mid', 'genre_small', 'buy_history_price']

In [7]:
print(len(raw.columns))
print(len(sparse_features))
print(len(dense_features))
print(len(unnecessary_features))

51
36
9
6


In [8]:
raw.drop(columns=unnecessary_features, inplace=True)

In [9]:
raw['label'] = 1

In [10]:
tmp = time.time()
data_dict = raw.set_index(keys=['sa_id', 'album_id']).T.to_dict('list')
print(time.time() - tmp)

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


958.6114776134491


In [11]:
user_col = ['sa_id', 'buy_seg', 'buy_tot']
cont_col = ['album_id', 'release_date', 'run_time', 'movie_meta_price', 'i30_meta_price',
                'amt_1_4', 'amt_2_4', 'amt_3_4', 'amt_4_4'] + [str(i) for i in range(num_genre)]

* all data -> user_table & content_table

In [12]:
user_tab = raw[user_col]
cont_tab = raw[cont_col]

* duplicate rows 제거

In [13]:
user_uni_tab = user_tab.loc[list(user_tab['sa_id'].drop_duplicates().index)]
cont_uni_tab = cont_tab.loc[list(cont_tab['album_id'].drop_duplicates().index)]

In [14]:
album_id_list = list(cont_uni_tab['album_id'])

* original method

In [15]:
def get_train_instances(train, num_negatives, album_id_list):
    '''
    설명 : train matrix를 받아서 그 안에 있는 user, item의 조합 1개당 num_negatives의 수만큼 negative sample을 만드는 함수.
    '''
    user_input, item_input, labels = [],[],[]
    
    train_keys = train.keys()
#     train_items = train.items()
    for (u, i) in tqdm.tqdm(train_keys):       
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        
        # negative instances
        for _ in range(num_negatives):
            j = random.randint(0, len(album_id_list) - 1)
            album = album_id_list[j]
            while (u, album) in train_keys :
                j = random.randint(0, len(album_id_list) - 1)
                album = album_id_list[j]
            user_input.append(u)
            item_input.append(album)
            labels.append(0)
#             output_dict[(u, j)] = 0
            
        
    return user_input, item_input, labels

In [16]:
user_input, item_input, labels = get_train_instances(data_dict, 2, album_id_list)

100%|██████████| 7786135/7786135 [00:40<00:00, 194424.26it/s]


In [17]:
final_df = pd.DataFrame({'sa_id': user_input, 'album_id':item_input, 'label':labels})

In [18]:
merge_df = pd.merge(final_df, user_uni_tab, how='inner', on='sa_id')

In [19]:
merge_df_2 = pd.merge(merge_df, cont_uni_tab, how='inner', on='album_id')

In [20]:
with open('../../Data/movie_201118_table_6_2.pickle', 'wb') as f:
    pickle.dump(merge_df_2, f)